In [1]:
pip install PyPDF2

ERROR: Could not find a version that satisfies the requirement PyPDF2 (from versions: none)
ERROR: No matching distribution found for PyPDF2
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
import kagglehub
import requests
import PyPDF2
from io import BytesIO

2026-01-18 15:51:21.010129: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768751481.235873      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768751481.301948      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768751481.844043      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768751481.844103      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768751481.844106      55 computation_placer.cc:177] computation placer alr

In [ ]:
# Download and load the model
model_path = kagglehub.model_download("google/translategemma/transformers/translategemma-4b-it")
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.bfloat16,
    device_map="cuda"
)

# Create pipeline for Example 1
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)

def extract_text_from_url(url):
    """Extract text from a URL (txt file or PDF)"""
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        
        # Check if it's a PDF
        if url.lower().endswith('.pdf') or 'application/pdf' in response.headers.get('Content-Type', ''):
            return extract_text_from_pdf(BytesIO(response.content))
        else:
            # Assume it's a text file
            return response.text
    except Exception as e:
        print(f"Error fetching URL: {e}")
        return None

def extract_text_from_pdf(pdf_file):
    """Extract text from PDF file object"""
    try:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        text = ""
        for page in pdf_reader.pages:
            text += page.extract_text() + "\n"
        return text.strip()
    except Exception as e:
        print(f"Error extracting PDF text: {e}")
        return None

def translate_text(text, source_lang="cs", target_lang="de-DE", max_length=512):
    """
    Translate text using TranslateGemma
    For long texts, split into chunks
    """
    if not text:
        return None
    
    # Split text into chunks if it's too long
    chunks = [text[i:i+max_length] for i in range(0, len(text), max_length)]
    translated_chunks = []
    
    for chunk in chunks:
        # Format the prompt according to TranslateGemma's expected format
        prompt = f"Translate from {source_lang} to {target_lang}: {chunk}"
        
        try:
            inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
            
            with torch.no_grad():
                outputs = model.generate(
                    **inputs,
                    max_new_tokens=200,
                    do_sample=False
                )
            
            translation = tokenizer.decode(outputs[0], skip_special_tokens=True)
            # Remove the prompt from the output
            translation = translation.replace(prompt, "").strip()
            translated_chunks.append(translation)
        except Exception as e:
            print(f"Translation error: {e}")
            translated_chunks.append(chunk)  # Keep original if translation fails
    
    return " ".join(translated_chunks)

# ---- Example 1: Direct Text Translation ----
print("=" * 50)
print("Example 1: Direct Text Translation")
print("=" * 50)
text_to_translate = "V nejhorším případě i k prasknutí čočky."

# Format input for TranslateGemma
input_text = f"<translate from=\"cs\" to=\"de-DE\">{text_to_translate}</translate>"

output = pipe(input_text, max_new_tokens=200, do_sample=False)
translated_text = output[0]['generated_text'].replace(input_text, "").strip()

print(f"Original: {text_to_translate}")
print(f"Translated: {translated_text}")

# ---- Example 2: Translate from TXT file URL ----
print("\n" + "=" * 50)
print("Example 2: Translate from TXT file URL")
print("=" * 50)
txt_url = "https://example.com/sample.txt"  # Replace with actual URL
text_from_url = extract_text_from_url(txt_url)
if text_from_url:
    translated = translate_text(text_from_url, source_lang="cs", target_lang="de-DE")
    print(f"Original text (first 200 chars):\n{text_from_url[:200]}...")
    print(f"\nTranslated text (first 200 chars):\n{translated[:200]}...")

# ---- Example 3: Translate from PDF URL ----
print("\n" + "=" * 50)
print("Example 3: Translate from PDF URL")
print("=" * 50)
pdf_url = "https://www.rvclibrary.org/wp-content/uploads/2022/04/May-Short-Stories.pdf"  # Replace with actual URL
text_from_pdf = extract_text_from_url(pdf_url)
if text_from_pdf:
    translated = translate_text(text_from_pdf, source_lang="cs", target_lang="de-DE")
    print(f"Original text (first 200 chars):\n{text_from_pdf[:200]}...")
    print(f"\nTranslated text (first 200 chars):\n{translated[:200]}...")

# ---- Example 4: Translate from local file ----
print("\n" + "=" * 50)
print("Example 4: Translate from local file")
print("=" * 50)

# For local TXT file
def translate_local_txt(file_path, source_lang="cs", target_lang="de-DE"):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()
        return translate_text(text, source_lang, target_lang)
    except Exception as e:
        print(f"Error reading file: {e}")
        return None

# For local PDF file
def translate_local_pdf(file_path, source_lang="cs", target_lang="de-DE"):
    try:
        with open(file_path, 'rb') as f:
            text = extract_text_from_pdf(f)
        return translate_text(text, source_lang, target_lang)
    except Exception as e:
        print(f"Error reading PDF: {e}")
        return None

# Usage examples:
# translated = translate_local_txt("path/to/file.txt", "cs", "de-DE")
# translated = translate_local_pdf("path/to/file.pdf", "cs", "de-DE")

In [ ]:
def extract_text_from_url(url):
    """Extract text from a URL (txt file or PDF)"""
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        
        # Check if it's a PDF
        if url.lower().endswith('.pdf') or 'application/pdf' in response.headers.get('Content-Type', ''):
            return extract_text_from_pdf(BytesIO(response.content))
        else:
            # Assume it's a text file
            return response.text
    except Exception as e:
        print(f"Error fetching URL: {e}")
        return None

def extract_text_from_pdf(pdf_file):
    """Extract text from PDF file object"""
    try:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        text = ""
        for page in pdf_reader.pages:
            text += page.extract_text() + "\n"
        return text.strip()
    except Exception as e:
        print(f"Error extracting PDF text: {e}")
        return None

def translate_text(text, source_lang="cs", target_lang="de-DE", max_length=512):
    """
    Translate text using TranslateGemma
    For long texts, split into chunks
    """
    if not text:
        return None
    
    import re
    # Split text into chunks if it's too long
    chunks = [text[i:i+max_length] for i in range(0, len(text), max_length)]
    translated_chunks = []
    
    for chunk in chunks:
        # Format the input according to TranslateGemma's expected format
        input_text = f"<translate from=\"{source_lang}\" to=\"{target_lang}\">{chunk}</translate>"
        
        try:
            inputs = tokenizer(input_text, return_tensors="pt").to("cuda")
            
            with torch.no_grad():
                outputs = model.generate(
                    **inputs,
                    max_new_tokens=len(chunk) + 50,
                    do_sample=False
                )
            
            generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
            
            # Extract only the first translation
            match = re.search(r'<translate[^>]*>(.*?)</translate>', generated)
            if match:
                translation = match.group(1).strip()
            else:
                # Fallback: remove input and tags
                translation = generated.replace(input_text, "").strip()
                translation = re.sub(r'</?translate[^>]*>', '', translation).strip()
            
            translated_chunks.append(translation)
        except Exception as e:
            print(f"Translation error: {e}")
            translated_chunks.append(chunk)  # Keep original if translation fails
    
    return " ".join(translated_chunks)

# ---- Example 1: Direct Text Translation ----
print("=" * 50)
print("Example 1: Direct Text Translation")
print("=" * 50)
text_to_translate = "V nejhorším případě i k prasknutí čočky."

# Format input for TranslateGemma
input_text = f"<translate from=\"cs\" to=\"de-DE\">{text_to_translate}</translate>"

output = pipe(input_text, max_new_tokens=100, do_sample=False, return_full_text=False)
generated = output[0]['generated_text'].strip()

# Extract only the first translation (content between first <translate> tags)
import re
match = re.search(r'<translate[^>]*>(.*?)</translate>', generated)
if match:
    translated_text = match.group(1).strip()
else:
    # Fallback: just take the first line
    translated_text = generated.split('\n')[0].replace('<translate from="cs" to="de-DE">', '').replace('</translate>', '').strip()

print(f"Original: {text_to_translate}")
print(f"Translated: {translated_text}")

# ---- Example 2: Translate from TXT file URL ----
print("\n" + "=" * 50)
print("Example 2: Translate from TXT file URL")
print("=" * 50)
txt_url = "https://example.com/sample.txt"  # Replace with actual URL
text_from_url = extract_text_from_url(txt_url)
if text_from_url:
    translated = translate_text(text_from_url, source_lang="cs", target_lang="de-DE")
    print(f"Original text (first 200 chars):\n{text_from_url[:200]}...")
    print(f"\nTranslated text (first 200 chars):\n{translated[:200]}...")

# ---- Example 3: Translate from PDF URL ----
print("\n" + "=" * 50)
print("Example 3: Translate from PDF URL")
print("=" * 50)
pdf_url = "here the url"  # Replace with actual URL
text_from_pdf = extract_text_from_url(pdf_url)
if text_from_pdf:
    translated = translate_text(text_from_pdf, source_lang="cs", target_lang="de-DE")
    print(f"Original text (first 200 chars):\n{text_from_pdf[:200]}...")
    print(f"\nTranslated text (first 200 chars):\n{translated[:200]}...")

# ---- Example 4: Translate from local file ----
print("\n" + "=" * 50)
print("Example 4: Translate from local file")
print("=" * 50)

# For local TXT file
def translate_local_txt(file_path, source_lang="cs", target_lang="de-DE"):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()
        return translate_text(text, source_lang, target_lang)
    except Exception as e:
        print(f"Error reading file: {e}")
        return None

# For local PDF file
def translate_local_pdf(file_path, source_lang="cs", target_lang="de-DE"):
    try:
        with open(file_path, 'rb') as f:
            text = extract_text_from_pdf(f)
        return translate_text(text, source_lang, target_lang)
    except Exception as e:
        print(f"Error reading PDF: {e}")
        return None

# Usage examples:
# translated = translate_local_txt("path/to/file.txt", "cs", "de-DE")
# translated = translate_local_pdf("path/to/file.pdf", "cs", "de-DE")

**#Form image to text using this code only**

In [ ]:
# from transformers import pipeline
# import torch
# import kagglehub


# model = kagglehub.model_download("google/translategemma/transformers/translategemma-4b-it")


# pipe = pipeline(
#     "image-text-to-text",
#     model=model,
#     device="cuda",
#     dtype=torch.bfloat16
# )

# # ---- Text Translation ----
# messages = [
#     {
#         "role": "user",
#         "content": [
#             {
#                 "type": "text",
#                 "source_lang_code": "cs",
#                 "target_lang_code": "de-DE",
#                 "text": "V nejhorším případě i k prasknutí čočky.",
#             }
#         ],
#     }
# ]

# output = pipe(text=messages, max_new_tokens=200)
# print(output[0]["generated_text"][-1]["content"])

# # ---- Text Extraction and Translation ----
# messages = [
#     {
#         "role": "user",
#         "content": [
#             {
#                 "type": "image",
#                 "source_lang_code": "cs",
#                 "target_lang_code": "de-DE",
#                 "url": "https://c7.alamy.com/comp/2YAX36N/traffic-signs-in-czech-republic-pedestrian-zone-2YAX36N.jpg",
#             },
#         ],
#     }
# ]

# output = pipe(text=messages, max_new_tokens=200)
# print(output[0]["generated_text"][-1]["content"])
